In [1]:
!pip install torch transformers librosa

In [12]:
import os
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [3]:
from transformers import Wav2Vec2Model

# Laden des Modells
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-xls-r-300m")

# Anzahl der Transformer-Schichten im Modell
num_layers = model.config.num_hidden_layers

print(f"Die Anzahl der Schichten im Modell ist: {num_layers}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Die Anzahl der Schichten im Modell ist: 24


In [19]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Setzen Sie den Pfad zu dem Verzeichnis, das Ihre MP3-Dateien enthält
directory_path = '/content/drive/My Drive/NLP/clips__test/gut_684dd9cf-2844-407b-9a7f-12e7b559773f'

print(len(os.listdir(directory_path)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
358


In [20]:
os.listdir(directory_path)[:10]

['cdfaa36ec13e3522214c57fa7de8c63274b2094593de3e37e7c5d6916d8890ed.mp3',
 'cd69df84b1cf478a0e771f970f36372187f1688b03ded0075c3827b4c3592ee2.mp3',
 'cd2a5c79dc56f2dcd5c071d1513acb213e65903ddb90d2ddd937ae331d06cb28.mp3',
 'cba8098f9dd8f3bc3e9b997506e00b858a539a6ef7e41dbf6e24686372384567.mp3',
 'cc63f1f361664c0be64f33ef32b9bec451bb8a44c352c6bb552c470571f798b4.mp3',
 'cc78a0dafea00fd9d7f83f9f327ae3de3f8a450ae15d3f98a7ab6008e64c8d34.mp3',
 'cc9139f276eadf10fbae7be5f49d4eae083babc67fb440943ab71b329786b9d0.mp3',
 'cc39409f8177f39fd61f35b31017b95bd3e11aa556f3b27080942e5915298d90.mp3',
 'cbebd2c4dd597ac286f851dffea5aaedf044918b5a9b5e70acb4e02eee512fc5.mp3',
 'cc370141daf4a2a665c3663fbd3d12ed94edd842ad1fb5f3727e3e3af97afc7c.mp3']

In [ ]:
import os
import librosa
import torch
from tqdm import tqdm
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

# Gerät festlegen, GPU wenn verfügbar, sonst CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-xls-r-300m")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-xls-r-300m", output_hidden_states=True)
model.to(device)  # Modell auf das festgelegte Gerät verschieben

def load_audio_files(directory, layer_indices=[-1]):
    """Lädt alle MP3-Dateien im angegebenen Verzeichnis und extrahiert die Repräsentationen aus den spezifizierten Schichten."""
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith(".mp3"):
            file_path = os.path.join(directory, filename)
            audio, sr = librosa.load(file_path, sr=16000)
            input_values = feature_extractor(audio, return_tensors="pt", sampling_rate=sr).input_values
            input_values = input_values.to(device)  # Eingabedaten auf das Gerät verschieben
            with torch.no_grad():
                outputs = model(input_values)
                for index in layer_indices:
                    hidden_states = outputs.hidden_states[index]
                    # Erstellt einen Unterordner im selben Verzeichnis für jede Schicht, falls nicht vorhanden
                    layer_dir = os.path.join(directory, f"layer_{index}")
                    os.makedirs(layer_dir, exist_ok=True)
                    save_path = os.path.join(layer_dir, f"{os.path.splitext(filename)[0]}_layer_{index}.npy")
                    np.save(save_path, hidden_states.cpu().numpy())
                    #print(f"Verarbeitet und gespeichert: {filename} in {save_path}")


load_audio_files(directory_path, layer_indices=[0, 5, 10, 15, 20, 24])